# Neural Machine Translation using seq2seq

In [2]:
import nltk
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import seaborn as sb
import pandas as pd
import numpy as np
import re
import os
import wget
import urllib
import sentpiece as sp
import csv
import urllib3
import zipfile
import shutil

C:\Users\5CG7092POZ\Anaconda3\envs\tf2.3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\5CG7092POZ\Anaconda3\envs\tf2.3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\5CG7092POZ\Anaconda3\envs\tf2.3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\5CG7092POZ\Anaconda3\envs\tf

## 1. Character-Level Neural Machine Translation using Functional API using fra-eng dataset
- 시퀀스-투-시퀀스(seq-to-seq)는 입력된 시퀀스로부터 다른 도메인의 시퀀스를 출력하는 다양한 분야에서 사용되는 모델입니다. 예를 들어 챗봇(Chatbot)과 기계 번역(Machine Translation)이 그러한 대표적인 예인데, 입력 시퀀스와 출력 시퀀스를 각각 질문과 대답으로 구성하면 챗봇으로 만들 수 있고, 입력 시퀀스와 출력 시퀀스를 각각 입력 문장과 번역 문장으로 만들면 번역기로 만들 수 있습니다. 그 외에도 내용 요약(Text Summarization), STT(Speech to Text) 등에서 쓰일 수 있습니다.

In [3]:
# url ="http://www.manythings.org/anki/fra-eng.zip"
# file = "fra-eng.zip"
# cur_dir = os.getcwd()
# file_dir = os.path.join(cur_dir, file)

# shutil.copyfileobj(urllib3.PoolManager().request("GET", url, preload_content=False), open(file_dir, "wb"))

# zipfile.ZipFile(file_dir, "r").extractall(cur_dir)

In [4]:
data = pd.read_csv("fra.txt", names=["src", "tar", "CC"], sep="\t")
data = data[["src", "tar"]]
data = data.sample(60000, random_state=777) # 6만개만 저장
data["src"] = data["src"].str.lower()
data["tar"] = data["tar"].str.lower()

![seq2seq](https://wikidocs.net/images/page/24996/%EC%9D%B8%EC%BD%94%EB%8D%94%EB%94%94%EC%BD%94%EB%8D%94%EB%AA%A8%EB%8D%B8.PNG)
- seq2seq는 크게 두 개로 구성된 아키텍처로 구성되는데, 바로 인코더와 디코더입니다. 인코더는 입력 문장의 모든 단어들을 순차적으로 입력받은 뒤에 마지막에 이 모든 단어 정보들을 압축해서 하나의 벡터로 만드는데, 이를 컨텍스트 벡터(context vector)라고 합니다. 입력 문장의 정보가 하나의 컨텍스트 벡터로 모두 압축되면 인코더는 컨텍스트 벡터를 디코더로 전송합니다. 디코더는 컨텍스트 벡터를 받아서 번역된 단어를 한 개씩 순차적으로 출력합니다.
- 디코더는 초기 입력으로 문장의 시작을 의미하는 심볼 `<sos>`가 들어갑니다. 디코더는 `<sos>`가 입력되면, 다음에 등장할 확률이 높은 단어를 예측합니다. 첫번째 시점(time step)의 디코더 RNN 셀은 다음에 등장할 단어로 je를 예측하였습니다. 첫번째 시점의 디코더 RNN 셀은 예측된 단어 je를 다음 시점의 RNN 셀의 입력으로 입력합니다. 그리고 두번째 시점의 디코더 RNN 셀은 입력된 단어 je로부터 다시 다음에 올 단어인 suis를 예측하고, 또 다시 이것을 다음 시점의 RNN 셀의 입력으로 보냅니다. 디코더는 이런 식으로 기본적으로 다음에 올 단어를 예측하고, 그 예측한 단어를 다음 시점의 RNN 셀의 입력으로 넣는 행위를 반복합니다. 이 행위는 문장의 끝을 의미하는 심볼인 `<eos>`가 다음 단어로 예측될 때까지 반복됩니다. 지금 설명하는 것은 테스트 과정 동안의 이야기입니다.

In [5]:
chars_src = set([char for sent in data["src"] for char in sent])
chars_tar = set([char for sent in data["tar"] for char in sent])

char2idx_src = {}
# char2idx_src["UNK"] = 1
char2idx_src.update({char:idx+1 for idx, char in enumerate(chars_src)})
idx2char_src = {value:key for key, value in char2idx_src.items()}

char2idx_tar = {}
# char2idx_tar["UNK"] = 1
char2idx_tar["<SOS>"] = 1
char2idx_tar["<EOS>"] = 2
char2idx_tar.update({char:idx+3 for idx, char in enumerate(chars_tar)})
idx2char_tar = {value:key for key, value in char2idx_tar.items()}

- 정상적으로 정수 인코딩이 수행된 것을 볼 수 있습니다. 아직 정수 인코딩을 수행해야 할 데이터가 하나 더 남았습니다. 디코더의 예측값과 비교하기 위한 실제값이 필요합니다. 그런데 이 실제값에는 시작 심볼에 해당되는 `<sos>`가 있을 필요가 없습니다. 이해가 되지 않는다면 이전 페이지의 그림으로 돌아가 Dense와 Softmax 위에 있는 단어들을 다시 보시기 바랍니다. 그래서 이번에는 정수 인코딩 과정에서 `<sos>`를 제거합니다. 즉, 모든 프랑스어 문장의 맨 앞에 붙어있는 '\t'를 제거하도록 합니다

In [6]:
enc_input = data["src"].apply(lambda x : [2] + [char2idx_src[char] for char in x] + [3]).tolist()
dec_input = data["tar"].apply(lambda x : [2] + [char2idx_tar[char] for char in x] + [3]).tolist()
dec_true = data["tar"].apply(lambda x : [char2idx_tar[char] for char in x] + [3]).tolist()

max_len_enc = np.max([len(sent) for sent in enc_input])
max_len_dec = np.max([len(sent) for sent in dec_input])

enc_input = tf.keras.preprocessing.seq.pad_seqs(enc_input, padding="post", maxlen=max_len_enc)
dec_input = tf.keras.preprocessing.seq.pad_seqs(dec_input, padding="post", maxlen=max_len_dec)
dec_true = tf.keras.preprocessing.seq.pad_seqs(dec_true, padding="post", maxlen=max_len_dec)

enc_input = tf.keras.utils.to_categorical(enc_input)
dec_input = tf.keras.utils.to_categorical(dec_input)
dec_true = tf.keras.utils.to_categorical(dec_true)

![rnn](https://wikidocs.net/images/page/24996/rnn%EA%B7%BC%ED%99%A9.PNG)
- 이미 RNN에 대해서 배운 적이 있지만, 다시 복습을 해보도록 하겠습니다. 하나의 RNN 셀은 각각의 시점(time step)마다 두 개의 입력을 받습니다. (이해가 되지 않는다면, RNN 챕터를 다시 참고하세요.)
- 현재 시점(time step)을 t라고 할 때, RNN 셀은 t-1에서의 은닉 상태와 t에서의 입력 벡터를 입력으로 받고, t에서의 은닉 상태를 만듭니다. 이때 t에서의 은닉 상태는 바로 위에 또 다른 은닉층이나 출력층이 존재할 경우에는 위의 층으로 보내거나, 필요없으면 값을 무시할 수 있습니다. 그리고 RNN 셀은 다음 시점에 해당하는 t+1의 RNN 셀의 입력으로 현재 t에서의 은닉 상태를 입력으로 보냅니다.
- RNN 챕터에서도 언급했지만, 이런 구조에서 현재 시점 t에서의 은닉 상태는 과거 시점의 동일한 RNN 셀에서의 모든 은닉 상태의 값들의 영향을 누적해서 받아온 값이라고 할 수 있습니다. 그렇기 때문에 앞서 우리가 언급했던 컨텍스트 벡터는 사실 인코더에서의 마지막 RNN 셀의 은닉 상태값을 말하는 것이며, 이는 입력 문장의 모든 단어 토큰들의 정보를 요약해서 담고있다고 할 수 있습니다.
- 디코더는 인코더의 마지막 RNN 셀의 은닉 상태인 컨텍스트 벡터를 첫번째 은닉 상태의 값으로 사용합니다. 디코더의 첫번째 RNN 셀은 이 첫번째 은닉 상태의 값과, 현재 t에서의 입력값인 `<sos>`로부터, 다음에 등장할 단어를 예측합니다.

#### Teacher Forcing
- 모델을 설계하기 전에 혹시 의아한 점은 없으신가요? 현재 시점의 디코더 셀의 입력은 오직 이전 디코더 셀의 출력을 입력으로 받는다고 설명하였는데 decoder_input이 왜 필요할까요?
- 훈련 과정에서는 이전 시점의 디코더 셀의 출력을 현재 시점의 디코더 셀의 입력으로 넣어주지 않고, 이전 시점의 실제값을 현재 시점의 디코더 셀의 입력값으로 하는 방법을 사용할 겁니다. 그 이유는 이전 시점의 디코더 셀의 예측이 틀렸는데 이를 현재 시점의 디코더 셀의 입력으로 사용하면 현재 시점의 디코더 셀의 예측도 잘못될 가능성이 높고 이는 연쇄 작용으로 디코더 전체의 예측을 어렵게 합니다. 이런 상황이 반복되면 훈련 시간이 느려집니다. 만약 이 상황을 원하지 않는다면 이전 시점의 디코더 셀의 예측값 대신 실제값을 현재 시점의 디코더 셀의 입력으로 사용하는 방법을 사용할 수 있습니다. 이와 같이 RNN의 모든 시점에 대해서 이전 시점의 예측값 대신 실제값을 입력으로 주는 방법을 교사 강요라고 합니다.

In [7]:
# inputs_enc = tf.keras.Input(shape=(None, n_chars_src+2))
inputs_enc = tf.keras.Input(shape=(max_len_enc, len(char2idx_src)+1), name="Input_enc")
# inputs_enc = tf.keras.Input(shape=(, len(char2idx_src)+1), name="Input_enc")
_, h_state, c_state = tf.keras.layers.LSTM(units=256, return_state=True, name="LSTM_enc")(inputs_enc)
# encoder_outputs도 같이 리턴받기는 했지만 여기서는 필요없으므로 이 값은 버림.
# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 바로 은닉 상태와 셀 상태.

# inputs_dec = tf.keras.Input(shape=(None, n_chars_tar+2))
inputs_dec = tf.keras.Input(shape=(max_len_dec, len(char2idx_tar)+1), name="Input_dec")
z= tf.keras.layers.LSTM(units=256, return_seqs=True, name="LSTM_dec")(inputs_dec, initial_state=[h_state, c_state])
# 디코더의 첫 상태를 인코더의 은닉 상태, 셀 상태로 합니다.
outputs_dec = tf.keras.layers.Dense(units=len(char2idx_tar)+1, activation="softmax", name="Dense")(z)

model = tf.keras.Model(inputs=[inputs_enc, inputs_dec], outputs=outputs_dec)

model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input_enc (InputLayer)          [(None, 241, 59)]    0                                            
__________________________________________________________________________________________________
Input_dec (InputLayer)          [(None, 307, 78)]    0                                            
__________________________________________________________________________________________________
LSTM_enc (LSTM)                 [(None, 256), (None, 323584      Input_enc[0][0]                  
__________________________________________________________________________________________________
LSTM_dec (LSTM)                 (None, 307, 256)     343040      Input_dec[0][0]                  
                                                                 LSTM_enc[0][1]        

In [8]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["categorical_accuracy"])

es = tf.keras.callbacks.EarlyStopping(monitor="val_loss", mode="auto", verbose=1, patience=4)
model_path = "fra-eng.h5"
mc = tf.keras.callbacks.ModelCheckpoint(filepath=model_path, monitor="val_categorical_accuracy", mode="auto", verbose=1, save_best_only=True)
hist = model.fit(x=[enc_input, dec_input], y=dec_true, batch_size=128, epochs=10, validation_split=0.2, callbacks=[es, mc])

Epoch 1/10
375/375 [==============================] - ETA: 0s - loss: 0.1733 - categorical_accuracy: 0.9475
Epoch 00001: val_categorical_accuracy improved from -inf to 0.94963, saving model to fra-eng.h5
375/375 [==============================] - 1709s 5s/step - loss: 0.1733 - categorical_accuracy: 0.9475 - val_loss: 0.1660 - val_categorical_accuracy: 0.9496
Epoch 2/10
375/375 [==============================] - ETA: 0s - loss: 0.1591 - categorical_accuracy: 0.9515
Epoch 00002: val_categorical_accuracy improved from 0.94963 to 0.95246, saving model to fra-eng.h5
375/375 [==============================] - 1759s 5s/step - loss: 0.1591 - categorical_accuracy: 0.9515 - val_loss: 0.1553 - val_categorical_accuracy: 0.9525
Epoch 3/10
375/375 [==============================] - ETA: 0s - loss: 0.1495 - categorical_accuracy: 0.9541
Epoch 00003: val_categorical_accuracy improved from 0.95246 to 0.95429, saving model to fra-eng.h5
375/375 [==============================] - 1773s 5s/step - loss: 0.1

KeyboardInterrupt: 

### (?) seq2seq 기계 번역기 동작시키기
- 앞서 seq2seq는 훈련할 때와 동작할 때의 방식이 다르다고 언급한 바 있습니다. 이번에는 입력한 문장에 대해서 기계 번역을 하도록 모델을 조정하고 동작시켜보도록 하겠습니다.
- 전체적인 번역 동작 단계를 정리하면 아래와 같습니다.
    1. 번역하고자 하는 입력 문장이 인코더에 들어가서 은닉 상태와 셀 상태를 얻습니다.
    2. 상태와 `<SOS>`를 디코더로 보냅니다.
    3. 디코더가 `<EOS>`가 나올 때까지 다음 문자를 예측하는 행동을 반복합니다.

In [8]:
model_path = "fra-eng.h5"
model = tf.keras.models.load_model(model_path)

- 우선 인코더를 정의합니다. encoder_inputs와 encoder_states는 훈련 과정에서 이미 정의한 것들을 재사용하는 것입니다. 이제 디코더를 설계해보겠습니다.

In [17]:
# h_state = model.layers[2].get_weights()[1]
# c_state = model.layers[2].get_weights()[2]

enc_model = tf.keras.Model(inputs=inputs_enc, outputs=[h_state, c_state])

# 이전 시점의 상태들을 저장하는 텐서
h_state_bef = tf.keras.Input(shape=(256,))
c_state_bef = tf.keras.Input(shape=(256,))
states_bef = [h_state_bef, c_state_bef]

z, h_state_aft, c_state_aft = tf.keras.layers.LSTM(units=256, return_seqs=True, return_state=True)(inputs_dec, initial_state=states_bef)
# 문장의 다음 단어를 예측하기 위해서 initial_state를 이전 시점의 상태로 사용. 이는 뒤의 함수 decode_seq()에 구현
states_aft = [h_state_aft, c_state_aft]
# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
outputs_dec = tf.keras.layers.Dense(units=len(char2idx_tar)+1, activation="softmax")(z)

# dec_model = tf.keras.Model(inputs=[inputs_dec] + states_bef, outputs=[outputs_dec] + states_aft)
dec_model = tf.keras.Model(inputs=[inputs_dec] + states_bef, outputs=[outputs_dec] + states_aft)

In [30]:
def decode_seq(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = enc_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, len(char2idx_tar)+1))
    target_seq[0, 0, char2idx_tar["<SOS>"]] = 1.

    stop_condition = False
    decoded_sent = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = dec_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2char_tar[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sent += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == "\n" or
           len(decoded_sent) > max_len_dec):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, len(char2idx_tar)+1))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sent

In [33]:
data

,src,tar
94598,please take care of yourself.,"prends soin de toi, s'il te plaît."
53718,where are your manners?,qu'as-tu fait de tes bonnes manières ?
140409,i broke a glass when i did the dishes.,j'ai cassé un verre en faisant la vaisselle.
101770,that was then and this is now.,c'était avant et c'est maintenant.
140504,i didn't even think that was possible.,je ne pensais pas que c'était possible.
...,...,...
13812,what's going on?,qu'y a-t-il ?
151400,i've been working here for several years.,je travaille ici depuis plusieurs années.
105922,i have no reason to lie to you.,je n'ai pas de raison de vous mentir.
132559,i can neither confirm nor deny this.,je ne peux ni confirmer ni infirmer ceci.


In [34]:
for seq_index in [13812]: # 입력 문장의 인덱스
    input_seq = enc_input[seq_index:seq_index + 1]
    decoded_sent = decode_seq(input_seq)
    print(35 * "-")
    print("입력 문장:", data["src"][seq_index])
    print("정답 문장:", data["tar"][seq_index][1:len(data["tar"][seq_index])-1]) # "\t"와 "\n"을 빼고 출력
    print("번역기가 번역한 문장:", decoded_sent[:len(decoded_sent)-1]) # "\n"을 빼고 출력

NameError: name 'decode_seq' is not defined

In [35]:
from nltk.translate.bleu_score import nltk.translate.bleu_score.corpus_bleu, SmoothingFunction
smooth_fn = SmoothingFunction()

SyntaxError: invalid syntax (<ipython-input-35-8d5618ddb0e7>, line 1)

In [5]:
import numpy as np
actual, predicted = list(), list()

for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sent = decode_seq(input_seq)
    
    actual.append([lines.tar[seq_index][1:len(lines.tar[seq_index])-1].split()])
    predicted.append(decoded_sent[:len(decoded_sent)-1].split())
                  
    print(35 * "-")
    print("입력 문장:", lines.src[seq_index])
    print(lines.src[seq_index].split())
    print("정답 문장:", lines.tar[seq_index][1:len(lines.tar[seq_index])-1]) # "\t"와 "\n"을 빼고 출력
    print(lines.tar[seq_index][1:len(lines.tar[seq_index])-1].split())
    print("번역기가 번역한 문장:", decoded_sent[:len(decoded_sent)-1]) # "\n"을 빼고 출력
    print(decoded_sent[:len(decoded_sent)-1].split())
    
    #print(actual)
    #print(predicted)
    print("BLEU-1: %f" % nltk.translate.bleu_score.corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0), smoothing_function = smooth_fn.method1))
    print("BLEU-2: %f" % nltk.translate.bleu_score.corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0), smoothing_function = smooth_fn.method1))
    print("BLEU-3: %f" % nltk.translate.bleu_score.corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0), smoothing_function = smooth_fn.method1))
    print("BLEU-4: %f" % nltk.translate.bleu_score.corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function = smooth_fn.method1))

NameError: name 'encoder_input' is not defined

In [6]:
import numpy as np
actual, predicted = list(), list()

for seq_index in range(0, len(encoder_input)): # 입력 문장의 인덱스
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sent = decode_seq(input_seq)
    
    actual.append([lines.tar[seq_index][1:len(lines.tar[seq_index])-1].split()])
    predicted.append(decoded_sent[:len(decoded_sent)-1].split())

    if seq_index < 10:
      print(35 * "-")
      print("입력 문장:", lines.src[seq_index])
      print(lines.src[seq_index].split())
      print("정답 문장:", lines.tar[seq_index][1:len(lines.tar[seq_index])-1]) # "\t"와 "\n"을 빼고 출력
      print(lines.tar[seq_index][1:len(lines.tar[seq_index])-1].split())
      print("번역기가 번역한 문장:", decoded_sent[:len(decoded_sent)-1]) # "\n"을 빼고 출력
      print(decoded_sent[:len(decoded_sent)-1].split())
    
      print("BLEU-1: %f" % nltk.translate.bleu_score.corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0), smoothing_function = smooth_fn.method1))
      print("BLEU-2: %f" % nltk.translate.bleu_score.corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0), smoothing_function = smooth_fn.method1))
      print("BLEU-3: %f" % nltk.translate.bleu_score.corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0), smoothing_function = smooth_fn.method1))
      print("BLEU-4: %f" % nltk.translate.bleu_score.corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function = smooth_fn.method1))
    
print("BLEU-1: %f" % nltk.translate.bleu_score.corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0), smoothing_function = smooth_fn.method1))
print("BLEU-2: %f" % nltk.translate.bleu_score.corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0), smoothing_function = smooth_fn.method1))
print("BLEU-3: %f" % nltk.translate.bleu_score.corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0), smoothing_function = smooth_fn.method1))
print("BLEU-4: %f" % nltk.translate.bleu_score.corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function = smooth_fn.method1))

NameError: name 'encoder_input' is not defined